In [20]:
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from wvaegan_gp.model_vector import Encoder, Decoder, Discriminator

In [17]:
cuda = torch.cuda.is_available()

OUTPUT_DIR = 'wvaegan_gp/results'
DATA_PATH = '../motor_dataset_20221018st'
N_CLASSES = 1
COORD_SIZE = 932  # 座標の数
LATENT_DIM = 1
done_epoch = 50000
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

rng = np.random.default_rng(0)
torques_npz = np.load(f"{DATA_PATH}/labels.npz")
coords_npz = np.load(f"{DATA_PATH}/coords.npz")
torques = torques_npz[torques_npz.files[0]]
torque_mean = torques_npz[torques_npz.files[2]]
torque_std = torques_npz[torques_npz.files[3]]
coords = coords_npz[coords_npz.files[0]]
coord_mean = coords_npz[coords_npz.files[1]]
coord_std = coords_npz[coords_npz.files[2]]

max_torque = torques.max()
min_torque = torques.min()

In [33]:
encoder = Encoder(LATENT_DIM, COORD_SIZE)
decoder = Decoder(LATENT_DIM, COORD_SIZE)
discriminator = Discriminator(COORD_SIZE)

E_PATH = f"{OUTPUT_DIR}/encoder_params_dim{LATENT_DIM}_vector_{done_epoch}"
G_PATH = f"{OUTPUT_DIR}/decoder_params_dim{LATENT_DIM}_vector_{done_epoch}"
D_PATH = f"{OUTPUT_DIR}/discriminator_params_dim{LATENT_DIM}_vector_{done_epoch}"

encoder.load_state_dict(torch.load(E_PATH, map_location=torch.device('cpu')))
encoder.eval()

decoder.load_state_dict(torch.load(G_PATH, map_location=torch.device('cpu')))
decoder.eval()

discriminator.load_state_dict(torch.load(D_PATH, map_location=torch.device('cpu')))
discriminator.eval()


Discriminator(
  (model): Sequential(
    (0): Linear(in_features=933, out_features=512, bias=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): Dropout(p=0.4, inplace=False)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [34]:
def save_coords_motor(gen_coords, labels, path):
    data_size = gen_coords.shape[0]
    fig, ax = plt.subplots(4,min(5, data_size//4), figsize=(12, 8), sharex=True, sharey=True)
    plt.subplots_adjust(hspace=0.4)
    for i in range(min(20, data_size)):
        coord = gen_coords[i][0]
        label = labels[i]
        
        ax[i%4, i//4].plot(coord[:136], coord[136:272])  # 磁石
        ax[i%4, i//4].plot(coord[272:272+184], coord[272+184:272+368])  # 穴
        # ax[i%4, i//4].plot(coord[272+368:272+368+146], coord[272+368+146:])  # 外枠

        torque = round(label.item(), 3)
        title = f'torque={torque}'
        ax[i%4, i//4].set_title(title)
    
    fig.savefig(path)
    plt.close()

In [35]:
def sample_image(epoch=None, data_num=12):
    labels = rng.uniform(min_torque, max_torque, size=(data_num, N_CLASSES))
    labels = Variable(FloatTensor(labels))

    z = Variable(FloatTensor(rng.standard_normal(size=(data_num, COORD_SIZE))))
    z = z.cuda()
    labels = labels.cuda()
    #データセットをencode
    mus, log_variances = encoder(z, labels)
    variances = torch.exp(log_variances * 0.5)
    Z_p = Variable(FloatTensor(rng.standard_normal(size=(data_num, LATENT_DIM))))
    Z = Z_p * variances + mus

    en_coords = decoder(Z, labels).cpu().detach().numpy()

    if cuda:
        labels = labels.cpu()
    labels = labels.detach().numpy()

    # 標準化を戻す
    en_coords_destandardized = en_coords*coord_std+coord_mean
    labels_destandardized = labels * torque_std + torque_mean

    save_coords_motor(en_coords_destandardized, labels_destandardized, "a.png")

In [36]:
sample_image()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)